In [6]:
import datetime

In [3]:
import tensorflow as tf
from tensorflow import keras 
print(tf.__version__)

1.9.0


In [4]:
keras.__version__

'2.1.6-tf'

In [13]:
def printRuntime():
    print(format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
    print("-"*16)

In [14]:
# every image is represented as a 28×28 array rather than a 1D array of size 784
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
printRuntime()

2019-10-16 12:17
----------------


In [15]:
X_train_full.shape

(60000, 28, 28)

In [16]:
X_test.shape

(10000, 28, 28)

In [18]:
# Create validation set
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [19]:
# for Fashion MNIST, we need the list of class names
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
print(class_names[y_train[0]])

Coat


In [21]:
# Specifying activation="relu" is equivalent to activation=keras.activations.relu
# Other activation functions are available in the keras.activations package, 
# See https://keras.io/activations/ for the full list.

# Creating the Model Using the Sequential API
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
printRuntime()

2019-10-16 12:32
----------------


In [ ]:
# Alternative way to specify the SAME model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
printRuntime()

- The first line creates a Sequential model. This is the simplest kind of Keras model, for neural networks that are just composed of a single stack of layers, con‐ nected sequentially. This is called the sequential API.  
- Next, we build the first layer and add it to the model. It is a Flatten layer whose role is simply to convert each input image into a 1D array: if it receives input data X, it computes X.reshape(-1, 1). This layer does not have any parameters, it is just there to do some simple preprocessing. Since it is the first layer in the model, you should specify the input_shape: this does not include the batch size, only the shape of the instances. Alternatively, you could add a keras.layers.InputLayer as the first layer, setting shape=[28,28].  
- Next we add a Dense hidden layer with 300 neurons. It will use the ReLU activation function. Each Dense layer manages its own weight matrix, containing all the connection weights between the neurons and their inputs. It also manages a vector of bias terms (one per neuron). When it receives some input data, it computes Equation 10-2.  
- Next we add a second Dense hidden layer with 100 neurons, also using the ReLU
activation function.  
- Finally, we add a Dense output layer with 10 neurons (one per class), using the softmax activation function (because the classes are exclusive).  

In [ ]:
# Model information

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.layers

In [24]:
model.layers[1].name

'dense_3'

In [26]:
hidden1 = model.layers[1]

In [27]:
# All the parameters of a layer can be accessed using get_weights() and set_weights() method  
weights, biases = hidden1.get_weights()
print(weights)
print(weights.shape)
print(biases)
print(biases.shape)

[[-0.0461628   0.04487149 -0.0047088  ... -0.04546276  0.04343
  -0.06276635]
 [ 0.00647467 -0.06553391 -0.0289762  ...  0.04988903 -0.05251116
  -0.02262454]
 [-0.01892196  0.01539844 -0.01335902 ...  0.04929199  0.01365545
  -0.06294311]
 ...
 [-0.07038769  0.023701    0.02298661 ...  0.02092859  0.00361724
  -0.05032304]
 [ 0.04325142  0.06139988  0.0169751  ... -0.0142545   0.04163028
   0.05830692]
 [-0.02738214 -0.05758239 -0.00317038 ... -0.02429691 -0.00640041
  -0.05087147]]
(784, 300)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

#### Compile the model

In [28]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
printRuntime()

2019-10-16 14:57
----------------


#### Reference
https://keras.io/losses/  
https://keras.io/optimizers/   
https://keras.io/metrics/  

#### Training and evaluating the model

In [29]:
# Args:
# input features (X_train) and the target classes (y_train)
# the number of epochs to train (default = 1)
# validation set (Optional): Keras will measure the loss and the extra metrics 
# on this set at the end of each epoch, which is very useful to see how well 
# the model really performs
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))
printRuntime()

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 4s 77us/step - loss: 0.7231 - acc: 0.7640 - val_loss: 0.5379 - val_acc: 0.8120
Epoch 2/30
55000/55000 [==============================] - 4s 64us/step - loss: 0.4887 - acc: 0.8293 - val_loss: 0.4511 - val_acc: 0.8380
Epoch 3/30
55000/55000 [==============================] - 3s 63us/step - loss: 0.4432 - acc: 0.8448 - val_loss: 0.4432 - val_acc: 0.8446
Epoch 4/30
55000/55000 [==============================] - 4s 66us/step - loss: 0.4143 - acc: 0.8543 - val_loss: 0.4108 - val_acc: 0.8566
Epoch 5/30
55000/55000 [==============================] - 4s 65us/step - loss: 0.3940 - acc: 0.8618 - val_loss: 0.3780 - val_acc: 0.8722
Epoch 6/30
55000/55000 [==============================] - 4s 64us/step - loss: 0.3784 - acc: 0.8669 - val_loss: 0.3866 - val_acc: 0.8686
Epoch 7/30
55000/55000 [==============================] - 4s 64us/step - loss: 0.3644 - acc: 0.8717 - val_loss: 0.3556 - val_acc: